In [113]:
import os
import pandas as pd

from sklearn.linear_model import LinearRegression
import numpy as np


In [114]:
DATA_PATH = './data'

train = pd.read_csv(os.path.join(DATA_PATH, 'train.csv'))
test = pd.read_csv(os.path.join(DATA_PATH, 'test.csv'))

# train_origin = train.copy()
# test_origin = test.copy()

print(f'train shape: {train.shape}')
print(f'test shape: {test.shape}')

train shape: (79023, 76)
test shape: (24353, 75)


In [115]:
test['emission'] = None
test = test[['ID_LAT_LON_YEAR_WEEK', 'emission']]

In [116]:
test

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,None
1,ID_-0.510_29.290_2022_01,None
2,ID_-0.510_29.290_2022_02,None
3,ID_-0.510_29.290_2022_03,None
4,ID_-0.510_29.290_2022_04,None
...,...,...
24348,ID_-3.299_30.301_2022_44,None
24349,ID_-3.299_30.301_2022_45,None
24350,ID_-3.299_30.301_2022_46,None
24351,ID_-3.299_30.301_2022_47,None


In [117]:
test_ids = test['ID_LAT_LON_YEAR_WEEK'].values
test_ids

array(['ID_-0.510_29.290_2022_00', 'ID_-0.510_29.290_2022_01',
       'ID_-0.510_29.290_2022_02', ..., 'ID_-3.299_30.301_2022_46',
       'ID_-3.299_30.301_2022_47', 'ID_-3.299_30.301_2022_48'],
      dtype=object)

In [118]:
for i in test_ids:
    # ID_LAT_LON
    id_lat_lon_idx = i.index('2022')
    id_lat_lon = i[:id_lat_lon_idx - 1]

    # week
    week = i[-2:]

    # train에서 같은 ID 찾기
    df = train.loc[train['ID_LAT_LON_YEAR_WEEK'].str.startswith(id_lat_lon)]
    df = df.loc[df['ID_LAT_LON_YEAR_WEEK'].str.endswith(week)]

    # 상관계수 구하기
    corr = df[['year', 'emission']].corr()
    if abs(corr['emission']['year']) >= 0.8:
        X = np.expand_dims(df.loc[:, 'year'].values, axis=1)
        y = np.expand_dims(df.loc[:, 'emission'].values, axis=1)
        
        lr = LinearRegression(fit_intercept=True)

        lr.fit(X, y) ## 모형 적합
        value = lr.predict([[2022]])
        
    else:
        value = df['emission'].mean()
    
    test.loc[test['ID_LAT_LON_YEAR_WEEK'] == i, 'emission'] = value

In [139]:
test

In [143]:
test['emission'] = test['emission'] * 1.06
test

,ID_LAT_LON_YEAR_WEEK,emission
0,ID_-0.510_29.290_2022_00,3.176613
1,ID_-0.510_29.290_2022_01,4.016319
2,ID_-0.510_29.290_2022_02,3.938143
3,ID_-0.510_29.290_2022_03,3.962625
4,ID_-0.510_29.290_2022_04,4.062005
...,...,...
24348,ID_-3.299_30.301_2022_44,28.725407
24349,ID_-3.299_30.301_2022_45,29.048892
24350,ID_-3.299_30.301_2022_46,29.252211
24351,ID_-3.299_30.301_2022_47,29.794027


In [142]:
test.to_csv('test2.csv')